<a href="https://colab.research.google.com/github/sgriesmer/njit-cad/blob/main/Training_DNABERT_Classifier_With_TFBS_Datasets_streamlined_sjg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

Installing Bio, transformers, genomic-benchmarks, and datasets packages.  The Bio package is from Biopython; transformers package for machine learning (pytorch, tensorflow); genomic-benchmarks and datasets from ML-Bioinfo-CEITEC.


In [ ]:
!pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 103.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 32.2 MB/s eta 0:00:00


Access TFBS dataset in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Run model on next 10 datasets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import Dataset, DatasetDict, load_metric
from transformers import TrainingArguments, Trainer
import numpy as np
import sys
import os

# initialize parameters

for fname in ["HaibA549Yy1cV0422111Etoh02UniPk.csv",
  "HaibA549Zbtb33V0422111Etoh02UniPk.csv",
  "HaibEcc1CtcfcV0416102Dm002p1hUniPk.csv",
  "HaibEcc1EraaV0416102Bpa1hUniPk.csv",
  "HaibEcc1EralphaaV0416102Est10nm1hUniPk.csv",
  "HaibEcc1EralphaaV0416102Gen1hUniPk.csv",
  "HaibEcc1Foxa1sc6553V0416102Dm002p1hUniPk.csv",
  "HaibEcc1GrV0416102Dex100nmUniPk.csv",
  "HaibEcc1Pol2V0416102Dm002p1hUniPk.csv",
  "HaibGm12878Atf2sc81188V0422111UniPk.csv"]:
  # "HaibGm12878Atf3Pcr1xUniPk.csv",
  # "HaibGm12878BatfPcr1xUniPk.csv",
  # "HaibGm12878Bcl11aPcr1xUniPk.csv",
  # "HaibGm12878Bcl3V0416101UniPk.csv",
  # "HaibGm12878Bclaf101388V0416101UniPk.csv",
  # "HaibGm12878Cebpbsc150V0422111UniPk.csv",
  # "HaibGm12878Ebf1sc137065Pcr1xUniPk.csv",
  # "HaibGm12878Egr1Pcr2xUniPk.csv",
  # "HaibGm12878Elf1sc631V0416101UniPk.csv",
  # "HaibGm12878Ets1Pcr1xUniPk.csv",
  # "HaibGm12878Foxm1sc502V0422111UniPk.csv",
  # "HaibGm12878GabpPcr2xUniPk.csv",
  # "HaibGm12878Irf4sc6059Pcr1xUniPk.csv",
  # "HaibGm12878Mef2aPcr1xUniPk.csv",
  # "HaibGm12878Mef2csc13268V0416101UniPk.csv",
  # "HaibGm12878Mta3sc81325V0422111UniPk.csv",
  # "HaibGm12878Nfatc1sc17834V0422111UniPk.csv",
  # "HaibGm12878Nficsc81335V0422111UniPk.csv",
  # "HaibGm12878NrsfPcr1xUniPk.csv",
  # "HaibGm12878P300Pcr1xUniPk.csv"]:

  dsname = "/content/drive/MyDrive/DNABERT/Datasets/tfbs/" + fname
  tfbs_dataset = pd.read_csv(dsname, sep=',')

# change standard error and output to saved output file

  stdpath = "/content/drive/MyDrive/DNABERT/output/" + fname.split(".")[0] + "/"

  try:
    os.makedirs(stdpath)
  except FileExistsError:
    pass

  stdpath_out = stdpath + "stdout.txt"
  stdpath_err = stdpath + "stderr.txt"

  sys.stdout = open(stdpath_out, 'w')
  sys.stderr = open(stdpath_err, 'w')

# print dataset shape and initial values

  print("tfbs dataset:", tfbs_dataset.shape, file=sys.stdout)
  print("tfbs dataset initial values:", tfbs_dataset.head(), file=sys.stdout)

# reformat input

  column_names = ["labels", "seq"]
  tfbs_dataset_res = pd.DataFrame(columns=column_names)

  j = 1
  for i in range(tfbs_dataset.count()['names']):
    name = tfbs_dataset['names'][i]
    pos_seq = tfbs_dataset['seq'][i]
    neg_seq = tfbs_dataset['neg_seq'][i]
    pos_label = 1
    neg_label = 0
    tfbs_dataset_res.loc[j] = [pos_label, pos_seq]
    tfbs_dataset_res.loc[j+1] = [neg_label, neg_seq]
    j+=2

  X = tfbs_dataset_res['seq']
  y = tfbs_dataset_res['labels']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# print size of training and test sets

  print("X_train shape:", X_train.shape, file=sys.stdout)
  print("y_train shape:", y_train.shape, file=sys.stdout)
  print("X_test shape:", X_test.shape, file=sys.stdout)
  print("y_test shape:", y_test.shape, file=sys.stdout)

# tokenization

  def kmers_stride1(s, k=6):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

# load pre-trained model

  model_cls = AutoModelForSequenceClassification.from_pretrained("armheb/DNA_bert_6", num_labels=2)
  params = list(model_cls.named_parameters())
  tokenizer = AutoTokenizer.from_pretrained("armheb/DNA_bert_6")

# reformat data to Hugging Face Dataset format from pandas

  ds_Xy_train = pd.concat([y_train, X_train], axis=1)
  ds_Xy_test = pd.concat([y_test, X_test], axis=1)

  Dataset_Xy_train = Dataset.from_pandas(ds_Xy_train)
  Dataset_Xy_test = Dataset.from_pandas(ds_Xy_test)
  Dataset_Xy_train, Dataset_Xy_test

  def tok_func(x): return tokenizer(" ".join(kmers_stride1(x["seq"])))

  Dataset_Xy_train_tok = Dataset_Xy_train.map(tok_func, batched=False)
  new_column = ["train"] * len(Dataset_Xy_train_tok)
  Dataset_Xy_train_tok = Dataset_Xy_train_tok.add_column("dset", new_column)

  Dataset_Xy_test_tok = Dataset_Xy_test.map(tok_func, batched=False)
  new_column = ["test"] * len(Dataset_Xy_test_tok)
  Dataset_Xy_test_tok = Dataset_Xy_test_tok.add_column("dset", new_column)

  dds = DatasetDict({
    'train': Dataset_Xy_train_tok,
    'test': Dataset_Xy_test_tok
  })

# switch to GPU

  if torch.cuda.device_count() > 0:
    model_cls.to('cuda')

# train model

  bs = 32
  epochs = 4
  lr = 8e-5

  args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

  def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

  trainer = Trainer(model_cls, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics)

  trainer.train();

# save model

  fpath = "/content/drive/MyDrive/DNABERT/Output_Models/" + fname.split(".")[0] + "/"
  print(fpath, file=sys.stdout)
  trainer.save_model(fpath)

# test model

  eval_preds = trainer.predict(dds['test'])

  print(eval_preds, file=sys.stdout)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at armheb/DNA_bert_6 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/77988 [00:00<?, ? examples/s]

Map:   0%|          | 0/25996 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.296700,0.253589,0.902293,0.898950
2,0.233500,0.270653,0.905716,0.906788
3,0.173200,0.287622,0.901216,0.904265


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.296700,0.253589,0.902293,0.898950
2,0.233500,0.270653,0.905716,0.906788
3,0.173200,0.287622,0.901216,0.904265
4,0.117900,0.407823,0.889945,0.895473
